Groundwater | Case Study

# Topic 1 : Introduction to the Groundwater course - The Limmat Vally Aquifer 

Dr. Xiang-Zhao Kong & Dr. Beatrice Marti & Louise Noel du Prat

In [ ]:
# Setting up the notebook
import sys
import os
import pandas as pd
import numpy as np

from IPython.display import display
import rasterio
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

# Load local modules
sys.path.append('../SUPPORT_REPO/src')
sys.path.append('../SUPPORT_REPO/src/scripts/scripts_exercises')
import climate_utils as cu
import print_images as du
import river_utils as ru
from data_utils import download_named_file
from map_utils import (
    display_groundwater_resources_map, 
    plot_model_area_map, 
    display_concessions_map
)
from progress_tracking import (
    create_perceptual_model_progress_tracker,
    create_section_completion_marker,
)

## Introducing the Case Study & Building a Perceptual Model
Most exercises as well as the final project in this course are based on case studies. Case studies are real-world scenarios that provide you with an opportunity to apply your knowledge and skills to solve complex groundwater problems. In this first year we choose the Limmat valley aquifer in Zurich, Switzerland. 

We will follow the actual steps a professional groundwater modeler might take to set up a numerical model to answer a specific question. In a first step, the modelers familiarize themselves with the Limmat valley aquifer and its hydrological and hydrogeological properties. We start out with a very generic perceptual model of the aquifer (see Figure 1), which we'll refine as we uncover new information. 

In [ ]:
du.display_image(
    image_filename='perceptual_model_00_initial.png', 
    image_folder='1_perceptual_model', 
    caption='Figure 1: Initial perceptual model which will be refined and used as a basis for a first long-term average water balance estimation.'
)

We first try to get an understanding of the hydrogeology of the Limmat valley aquifer. And then go through each water balance component and try to get a first rough understanding of the most important fluxes in the system. At a later stage, we will re-use and refine the perceptual model to set up a numerical model of the Limmat valley aquifer.

## The Limmat Valley Aquifer
The Limmat valley aquifer is a well-studied groundwater reservoir beneath the city of Zurich, Switzerland. Doppler and colleagues write, that it was formed during the last ice age, when the Lindt glacier retreated. The Limmat valley aquifer has no direct hydraulic connection to lake Zurich in the east where it is bound by impermeable lake sediments and moraine material. The aquifer is further confined in the north and south by the side morains of the Lindth glacier. Lateral inflow of groundwater from the hills in the north and south is to be expected. The groundwater body is further in connection with the river Sihl in the east and the river Limmat in the north. The hydraulic properties of the aquifer are higly heterogenic because of its complex geological history formed through various sedimentation and erosion events from the rivers Sihl and Limmat. [\[1, 2\]](#references)

In Figure 2 you see a prinscreen of the [GIS-browser](https://www.gis.zh.ch/) of the canton of Zurich [\[3\]](#references). The cyanide blue area in the center of the map shows the Limmat valley aquifer. The darker the color, the larger the thickness of the aquifer. The GIS-Browser is only available in German. You can use the translation feature of your browser to translate the page into your preferred language.

In [ ]:
du.display_image(
    image_filename='GIS-browser_canton_Zurich.png', 
    image_folder='1_perceptual_model',
    caption='Figure 2: Printscreen of the GIS-browser of the canton of Zurich displaying the cantonal groundwater map. Source: https://www.gis.zh.ch/, accessed 2024-05-01'
)

We now start working through the available information to refine our perceptual model of the Limmat valley aquifer. We will use the GIS-browser to refine our understanding of the geometry of the aquifer and how we want to represent it in the model. 

You can use the list below to track your progress through the perceptual model construction. 

In [ ]:
create_perceptual_model_progress_tracker()

## Aquifer Geometry
Most of the layers in the GIS-Browser of the Canton of Zurich are available to the general public for download (button "Datenbezug" in the upper right corner of the map pane). We use this feature and import the relevant parts of the map into our JupyterHub environment. 

### Bottom Topography
We have downloaded the aquifer thickness layer for you from the GIS-browser and clipped it to the area of interest (AOI) of the Limmat valley aquifer. Let's have a look at the bottom topography of the aquifer in Figure 3.

In [ ]:
# This function downloads the groundwater map of the canton of Zurich and saves 
# it to the folder applied_groundwater_modelling_data in your home directory. 
gw_map_path = download_named_file(
    name='groundwater_map_norm', 
    data_type='gis', 
)


In [ ]:
# Create and display the interactive map
interactive_map = display_groundwater_resources_map(
    gw_map_path, zoom_level=11, 
    map_title="Figure 3: Groundwater resources map of the canton of Zurich",)
interactive_map

We can use these contours of the aquifer thickness to build the bottom of the aquifer in our model later on.  

### Top Topography
The Limmat valley aquifer is unconfined. We can use the topography of the Limmat valley to represent the top of the aquifer. swisstopo provides digital elevation models (DEMs) of Switzerland in various resolutions. We use the 25m resolution DEM for our case study. We have downloaded the DEM from the [website of the Federal Office of Topography swisstopo](https://www.swisstopo.admin.ch/en/geodata/height/dhm25.html)[\[4\]](#references) and clipped it to the area of interest (AOI) of the Limmat valley aquifer (Figure 4). If you are interested to learn more about the DEM and how to clip it to the AOI, you can read the [processing_DEM.ipynb](../SUPPORT_REPO/src/scripts/scripts_limmat_data_preprocessing/processing_DEM.ipynb) notebook.

In [ ]:
dem_path = download_named_file(
    name='dem',
    data_type='gis'
)

In [ ]:
# Open the DEM file
with rasterio.open(dem_path) as dem_src:
    dem_data = dem_src.read(1)  # Read the first band
    transform = dem_src.transform
    height, width = dem_src.height, dem_src.width
    x_coords = [transform.c + i * transform.a for i in range(width)]
    y_coords = [transform.f + i * transform.e for i in range(height)]

# Use the clipped and masked DEM from the previous steps
fig = go.Figure(data=go.Heatmap(
    z=dem_data,
    x=x_coords,
    y=y_coords,
    colorscale='earth',
    zmin=0,
    colorbar=dict(title='Elevation (m)'), 
))

fig.update_layout(
    title='Figure 4: Elevation model DHM25 (©swisstopo) of the Limmat Valley (Clipped Area).',
    xaxis_title='Easting (m)',
    yaxis_title='Northing (m)',
    # Keep aspect ratio correct
    yaxis_scaleanchor="x",
    yaxis_scaleratio=1,
    xaxis_tickformat='d',
    yaxis_tickformat='d'
)
fig.update_layout(title_font_size=15)

fig.show()

The topography of the Limmat valley is quite flat, with a maximum elevation of about 400 m above sea level. As we use a rather coarse DEM resolution of 25 m, we can not see the small topographic features in the Limmat valley, like for example the river beds. As we would like to model the river-aquifer interaction, we need to find a suitable compromise between DEM (and model grid) resolution and computational capacities to represent the river beds in our model.  

River-aquifer interaction is predominantly driven by the stream bed elevation relative to the groundwater table. Therefore, an accurate representation of the streambed elevation in the numerical groundwater flow model is crucial. With a spatial resolution of 20 m, the DEM is not sufficient to represent the river beds in the Limmat valley aquifer. We will discuss the river-aquifer interaction in a later chapter, where we will also discuss how to represent the river beds in our model for different modelling purposes in the case studies. 

### Lateral Model Boundaries
The groundwater map of the Limmat valley aquifer shows that the aquifer is not hydraulically connected to the lake Zurich in the east, but that it is bounded by impermeable lake sediments and moraine material. We will neglect aquifer parts east of the river Sihl as it is not the focus of the model goal and we expect the recharge from the river to dominate the flow from the east. The aquifer is further confined in the north and south by the side morains of the Lindth glacier. Lateral inflow of groundwater from the hills in the north and south is to be expected. The most common approach is to use a water balance approach on the hill sides to estimate the sub-surface inflow from the hills. We'll address this in the chapter on [climate data](#climate). We do not consider the lateral inflow from the Sihl valley aquifer into the Limmat valley aquifer, as the inflow is rather small and parallel to the general flow direction in the Limmat valley. 

Should you have to estimate the lateral inflow from the Sihl valley aquifer, you can apply Darcy's law under the Dupuit assumption. In practice, the organization tasking you to produce a groundwater flow model will typically provide you with estimates of hydraulic conductivities from field measurements. As a first estimate, you can use the hydraulic conductivities from the literature (e.g. Freeze and Cherry, 1979, shown in Figure 5)[\[5\]](#references).   

In [ ]:
print(10**(-2)*3600*24)

In [ ]:
du.display_image(
    image_filename='hydraulic_conductivities_by_Freeze.png', 
    image_folder='1_perceptual_model',
    caption='Figure 5: Hydraulic conductivity ranges as presented in the book <<Groundwater>> by Freeze and Cherry (1979). The ranges are based on the hydraulic conductivity of unconsolidated sediments. Hydraulic conductivity is typically known only in approximate orders of magnitude.'
)


> 📚 **Theory reminder**: Applying Darcy's Law in Unconfined Aquifers: The Dupuit Approximation
>
> To apply Darcy's Law for calculating the total discharge through an unconfined aquifer, we often rely on a set of simplifying assumptions known as the *Dupuit approximation* (or Dupuit-Forchheimer assumption). These assumptions allow us to treat the complex three-dimensional flow problem as a simpler two-dimensional one by integrating flow over the entire aquifer thickness.
> 
> The key conditions that must be met for the Dupuit approximation to be valid are:
> 
> *   *The water table gradient is small:* The slope of the phreatic surface must be gentle. This is the most critical assumption, as it implies that flow is almost entirely horizontal.
> *   *Flow is horizontal:* It is assumed that the hydraulic head is constant with depth for any given vertical line. This means equipotential lines are vertical, and consequently, the flow lines are horizontal.
> *   *The hydraulic gradient is equal to the slope of the water table:* The slope of the water table is assumed to be the hydraulic gradient (`i`) for the entire saturated thickness of the aquifer.
> 
> When these conditions hold, we can use a modified form of Darcy's Law to calculate the discharge `Q` of the aquifer:
> 
> `Q = -K * h * B * (dh/dx)`
> 
> Where:
> *   `Q` is the discharge (e.g., m3/s)
> *   `K` is the hydraulic conductivity (e.g., m/s)
> *   `h` is the saturated thickness (m)
> *   `B` is the width of the aquifer perpendicular to the flow direction (m)
> *   `dh/dx` is the slope of the water table, which represents the hydraulic gradient.
> 
> In essence, the Dupuit approximation simplifies reality by ignoring the vertical components of flow within the aquifer, which is a reasonable simplification when the water table is nearly flat compared to the saturated thickness.

For the outflow, a fixed-head boundary is typically chosen. This requires a known groundwater table at the boundary, e.g. a monitoring piezometer. Another option is to use the water level of a receiving stream or lake as a fixed-head boundary condition. The outflow boundary should be placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition. For regional model, this is typically several hundred meters to several kilometers away from the area of interest. 

For our case, we choose the groundwater level at piezometer 481 (indicated with a pink arrow in Figure 6) as the fixed-head boundary condition. This relieves us of the need to implement the more complex aquifer geometry and river-aquifer interaction further downstream of the Limmat valley aquifer. We can use the isoline of equal groundwater levels from the GIS-browser to define the outflow boundary.

In [ ]:
du.display_image(
    image_filename='Detail_outflow.png', 
    image_folder='1_perceptual_model',
    caption='Figure 6: Detail of the outflow boundary condition at piezometer 481 (purple arrow). The groundwater level at this piezometer is used as a fixed-head boundary condition for the groundwater flow model. The outflow boundary is placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition.'
)

Note that the isolines of equal groundwater levels are not perfectly perpendicular to the flow direction but slightly curved at the aquifer boundaries, particularly well visible at the northern boundary. This indicates lateral groundwater inflow from the north.  

We can now make a rough estimate of the groundwater flux at the western outflow boundary of the Limmat valley aquifer using the Dupuit assumption. We'll use the two isolines of equal groundwater levels of 389 m a.s.l. and 391 m a.s.l. to estimate the hydraulic gradient at the western boundary of the Limmat valley aquifer. The distance between the two isolines can be measured in a GIS tool and is about 780 m, the thickness of the aquifer in that location is between 2 m and 10 m, and the aquifer width is approximately 1 km. We estimate the hydraulic conductivity to be around 10^-2 m/s so we can estimate the hydraulic gradient as follows:

In [ ]:
outflow_lower_bound = (391 - 389) / 780 * 2 * 1000 * 0.01 * 3600 * 24 # m3/day
outflow_upper_bound = (391 - 389) / 780 * 10 * 1000 * 0.01 * 3600 * 24 # m3/day

print(f"Estimated groundwater outflow at the western boundary of the Limmat valley aquifer: {outflow_lower_bound:.2f} m3/day to {outflow_upper_bound:.2f} m3/day")

We now have an understanding of the geometry of the aquifer. Let's simplify update our perceptual model with the information we have gathered so far: 

- The aquifer does not have a direct hydraulic connection to lake Zurich in the east. 
- It is probably heavily influenced by the river Sihl in the east and the river Limmat in the north.
- Since groundwater flow in river valleys generally follows the topography, we can assume that the groundwater flow in the Limmat valley aquifer is directed from south-east to west. This general flow direction is corroborated by the groundwater flow arrows which become visible when we zoom in on the GIS-browser map (visit the GIS browser to zoom in).
- The aquifer thickness is highly variable. To adequately represent the known geometry of the aquifer, a 3D model with a high spatial resolution is required. We will first opt for a 2D model with a simplified geometry for the case study work to keep the model lean and fast on JupyterHub. 
- The simplified geometry will show a larger thickness in the south-east with a gradual thinning towards the west. The thickness will be represented by a single layer with a variable thickness. We will use the thickness values from the GIS-browser to define the thickness of the aquifer in our model whereby we will have to map the isolines of groundwater thickness to the model grid (see Figure 7).  
- The aquifer extends boyond the boundaries of the city of Zurich. Since we are intersted mainly in applying our numerical experiment in the city region, this means that we will have to make assumptions about the outflow boundary of the aquifer.

In [ ]:
du.display_image(
    image_filename='perceptual_model_01_simple_geometry.png', 
    image_folder='1_perceptual_model',
    caption='Figure 7: Illustration of the refined perceptual model, including information about the aquifer shape.'
)

<div style="padding: 1em; margin: 1em 0; border-left: 3px solid #8e44ad; background-color: #f5eef8;">
<strong>🤔 Think about it:</strong><br>
Take a few minutes to explore the available layers in the GIS-browser of the canton of Zurich. What do you think are the most important layers for groundwater modeling? Why?
</div>

In [ ]:
create_section_completion_marker(1)  # Mark section 1 complete

## Climate Forcing
Climate data is used to estimate the areal recharge of the aquifer from infiltration of precipitation. It is further used to estimate the lateral inflow from the hills in the north and south of the Limmat valley aquifer. The lateral inflow is estimated using a water balance approach on the hill sides.

### Areal Recharge
We get climate data from the Swiss Federal Office of Meteorology and Climatology (MeteoSwiss). The data is available for viewing on [MeteoSwiss](https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year) website [\[6\]](#references) and was downloaded via the [opendata.swiss plattform](https://opendata.swiss/en/dataset/klimanormwerte) [\[7\]](#references). We made it available in the data directory of the zurich case study repository. 
The closest station to the Limmat valley aquifer is the Fluntern station. The data is available for the years 1991-2020. Let's have a look at the data. 

In [ ]:
data_path = os.path.abspath(
    os.path.join('case_study_zurich', 'data', 'climate'))
# Test if this is a valid path
if not os.path.exists(data_path):
    print(f"Path {data_path} does not exist.")

climate_norms = cu.read_climate_data(data_path, station_string='Fluntern')

# Inspect the climate norms
display(climate_norms)

> **🤔 Think about it**  
> We are looking at the climate data mostly to get an idea about groundwater recharge. In an aquifer like the Limmat valley aquifer, which is located in a densely populated area, we have to consider that the recharge is not only influenced by the climate but also by human activities. What do you think are the most important human activities that influence groundwater recharge in this area?

Note that we not only have temperature and precipitation data but a wide range of other climate variables which we can use to calculate potential evaporation. However, since the Limmat valley aquifer is located in a built-up area, we will have to make some assumptions about the land use and the resulting potential evaporation. Approximately 80% to 90% of the upper boundary of the aquifer is built-up, with only a small fraction of the area being green space or railway tracks. According to Qui and colleagues, urban evaporation in Swiss climate can amount to about 20% of the precipitation[\[8\]](@references), that would be around 200 mm/year for the Limmat valley aquifer. We will use this value as a first approximation for the potential evaporation in our model. 

Figure 8 below shows the monthly average temperature and precipitation for the Fluntern station. 

In [ ]:
plt, fig = cu.plot_climate_data(
    climate_norms, 
    custom_title="Figure 8: Climate data for the Fluntern station (source ).")
plt.show()

We now have first data concerning the fluxes into and out of the aquifer. We can now update our perceptual model with the information we have gathered so far:
- The average annual precipitation in the Limmat valley aquifer is 1108 mm/year. 
- We assume that the actual evaporation is about 20% of the precipitation, which would be around 200 mm/year (see Figure 8).
- Lehner suggests that between 5% and 15% of the precipitation infiltrates into the aquifer [\[9\]](@references). We will use a value of 10% as a first approximation, which would be around 110 mm/year.
- The rest of the precipitation runs off, via the rainwater drainage system, into the river Sihl and the river Limmat. That would be about 70% of the precipitation, which would be around 770 mm/year.

We update the perceptual model again in Figure 9. 

In [ ]:
du.display_image(
    image_filename='perceptual_model_02_atmospheric_fluxes.png', 
    image_folder='1_perceptual_model',
    caption='Figure 9: Adding first estimates of atmospheric forcing to perceptual model.'
)

TODO: Change figure, replace E and P with net recharge to the aquifer (R)

Once we know the area of the model domain, we can calculate the areal net recharge to the aquifer. 

### Lateral Inflow from Hills
The lateral inflow from the hills in the north and south of the Limmat valley aquifer is estimated using a water balance approach on the hill sides. We will use the same climate data as for the areal recharge to estimate the lateral inflow.

We can use the same approach as for the areal recharge to estimate the lateral inflow from the hills. We require the area of the hills in the north and south of the Limmat valley aquifer to calculate the lateral inflow. 

As a first approximation, we roughly delineate the area of the hills in the north and south of the Limmat valley aquifer using QGIS (see Figure 10). We can then use the climate data to estimate the lateral inflow from the hills. 

In [ ]:
du.display_image(
    image_filename='rough_first_manual_catchment_delineation_to_estimate_area_for_lateral_inflow.png', 
    image_folder='1_perceptual_model',
    caption='Figure 10: Rough first manual catchment delineation to estimate area for lateral inflow. The area is delineated using the height model of the area (here DHM200). The area is used to estimate the lateral inflow from the hills in the north and south of the Limmat valley aquifer. The delineation is not precise and should be refined in a later step. The area relevant for the lateral inflow estimation is 15 km2 in the south and 11 km2 in the north of the Limmat valley aquifer.'
)

Please note, in this very first iteration of the perceptual model, we use crude approximations. A proper catchment delineation would require a more detailed analysis of the topography and the hydrological conditions in the area. You find step-by-step instructions on how to delineate a catchment in many places on the internet, for example here: [https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation](https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation). 

We assume that the lateral inflow of groundwater into the aquifer is only about 10% of the precipitation in the area. Our first rough estimate of the lateral groundwater inflow from the hills to the north and south of the Limmat valley thus is:  

In [ ]:
inflow_south = 0.1 * 15e6  # m3/year
inflow_north = 0.1 * 11e6  # m3/year

print(f"Estimated lateral inflow from the south: {(inflow_south/1000000):.1f} 10^6 m³/year")
print(f"Estimated lateral inflow from the north: {(inflow_north/1000000):.1f} 10^6 m³/year")

> **Think about it:**. 
> What type of boundary condition would you use to represent the lateral inflow from the hills in a steady state numerical model? Why?  

TODO: Update the water balance figure with the lateral inflow from the hills


In [ ]:
create_section_completion_marker(2)  # Mark section 2 complete

## River-aquifer interaction

### River Geometry & Recharge
The river-aquifer interaction is an important component of the groundwater flow model. The geometry of the river and its interaction with the aquifer will influence the groundwater flow direction and the groundwater levels in the aquifer.

In practice, a modeler will use measured profiles of the river bed to represent the river geometry in the model. For this course, we do not have access to profiles of the river bed so we have to make some assumptions about the river geometry.

If you are based in Zurich, you can visit the rivers, for example on a bicycle tour. Visiting your model area is a great way to get a feel for the area and to understand the hydrological processes at play. If you don't have the opportunity to visit the area, you can use the GIS-browser or download selected tiles of the high-resolution elevation model to get an idea of the river geometry and ask a local project partner to help you build the perceptual model. 

#### Interactive Exploration of River-Aquifer Interaction

Figure 11 below provides an interactive tool to help you visualize the complex relationship between a river and an adjacent unconfined aquifer. It consists of two main parts:

1.  **Left Plot (Physical Cross-Section):** This shows a physical model of the river and the groundwater system. You can see the water level in the river (`H_riv`) and the water table in the aquifer (`H_aq`).
2.  **Right Plot (Flux Relationship):** This is a conceptual graph that shows the calculated flux (flow rate, `Q_riv`) between the river and the aquifer as a function of the aquifer's head. The red dot indicates the current state of the system shown on the left.

**How to Use This Figure:**

Use the two sliders below the plot to change the **Aquifer Head (`H_aq`)** and the **River Stage (`H_riv`)**. As you move the sliders, observe how the system changes in both plots. Pay close attention to the title, the equation, the direction of the flow (indicated by the arrow), and the position of the red dot on the flux curve.

**Scenarios to Explore:**

Try to create the following three fundamental conditions:

1.  **Gaining Stream:**
    *   **How to create it:** Set the aquifer head (`H_aq`) to be *higher* than the river stage (`H_riv`).
    *   **What to observe:** Water flows from the aquifer into the river. The flux (`Q_riv`) is negative (flow *out of* the groundwater). The head difference (`ΔH`) is between `H_aq` and `H_riv`. Notice on the right plot that the red dot is in the negative flux region.

2.  **Losing Stream (Connected):**
    *   **How to create it:** Set the river stage (`H_riv`) to be *higher* than the aquifer head (`H_aq`), but keep `H_aq` *above* the riverbed bottom (`R_bot`, the dashed brown line).
    *   **What to observe:** Water flows from the river into the aquifer. The flux is positive and its magnitude depends directly on the head difference (`ΔH`). As you lower `H_aq`, the flux increases. The red dot on the right plot moves down the sloped part of the curve.

3.  **Losing Stream (Disconnected):**
    *   **How to create it:** Lower the aquifer head (`H_aq`) until it is *below* the riverbed bottom (`R_bot`).
    *   **What to observe:** A yellow "Vadose Zone" appears between the riverbed and the water table, indicating they are no longer directly connected. The flow from the river now seeps downward at a constant, maximum rate. The flux no longer depends on `H_aq`. Notice that the head difference (`ΔH`) for the calculation is now between `H_riv` and `R_bot`. On the right plot, the red dot moves onto the vertical part of the curve, showing that the flux remains constant even if you continue to lower `H_aq`.

In [ ]:
ru.plot_river_aquifer_interaction(
    custom_title="Figure 11: Illustration of river-aquifer interaction and the corresponding flux vs. head relationship."
)

### River Discharge & River Water Levels
The federal office for the environment (FOEN) is the first address for hydrological data in Switzerland. You will find all surface water monitoring sites under [map.geo.admin.ch](https://map.geo.admin.ch) under layer *Hydrological gauaging stations* [\[10\]](#references) (see Figure 12). 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations.png', 
    image_folder='1_perceptual_model',
    caption='Figure 12: Hydrological gauging stations layer available at map.geo.admin.ch.' 
)

When you zoom in to the Limmat valley aquifer (Figure 13), you will find the gauging stations of the rivers Sihl and Limmat. A next gauging station on the river Limmat is located at the city of Baden, downstream of a run-by-the-river hydropower plant, and therefore not relevant for our study. The gauging station on the river Sihl is located at the city of Zurich, upstream of the confluence with the river Limmat. The gauging station on the river Limmat is located at the city of Zurich, downstream of the confluence with the river Sihl. 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations_closeup.png',
    image_folder='1_perceptual_model',
    caption='Figure 13: Locations of hydrological gauging stations near the Limmat valley aquifer.', 
)

A click on the gauaging station will bring you directly to the station site made available by the federal office for the environment (FOEN). The gauging station on the river Sihl is called [*Sihl - Zürich, Sihlhölzli*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2176) and has ID 2176 and the gauging station on the river Limmat is called [*Limmat - Zürich, Unterhard*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2099) and has ID 2099. The IDs are unique numeric station identifiers and typically required to retrieve data from data repositories or APIs. 

> **🤔 Think about it**  
> For surface water balancing, discharge is the important variable. However, when it comes to flooding or river-aquifer interaction, the water level is the more important variable. Why do you think that is?

From the station sites, we see current water levels, typically over the past 7 days but no water level dynamics. Thankfully, this data can be requested from FOEN. You will find a copy of the data in the data directory of the zurich case study repository. The data is available as a csv file. Please note that the data from the most recent years is provisional data and may be subject to change.

Let's have a look at the data (Figures 14 and 15 and table below). 


In [ ]:
# Define the path to river data
river_data_path = os.path.abspath(
    os.path.join('case_study_zurich', 'data', 'rivers'))

# Check if path exists
if not os.path.exists(river_data_path):
    print(f"Path {river_data_path} does not exist.")
else:
    try:
        # Plot the typical yearly evolution of river water levels
        fig, axes = ru.plot_yearly_river_levels(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020,
            figsize=(12, 8), 
            figure_number=14  # Set figure number for the plot
        )
        
        # Display the plot
        plt.show()
        
        # Get summary statistics
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")
        
    except ImportError as e:
        print(f"Plotting functionality not available: {e}")
        print("However, we can still analyze the data...")
        
        # Get summary statistics even without plotting
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")

# Save the river data summary to a CSV file
summary_df = np.save(
    os.path.join(river_data_path, 'river_data_summary.npy'),
    summary
)

No additional river gauging stations are maintained by the cantonal office for waste, water, energy and air (Amt für Abfall, Wasser, Energie und Luft (AWEL)) [\[11\]](#references). 

Let's bring the river water levels in context with the groundwater levels we read from the groundwater level map near the gauging stations in Figure 16. 

In [ ]:
# Sihl River Data
sihl_gw_mean = 400.8
sihl_gw_high = 403.5
sihl_river_mean = 412.347
sihl_river_high = 413.439
sihl_depth_mean = 0.3

# Limmat River Data
limmat_gw_mean = 399.5
limmat_gw_high = 400.5
limmat_river_mean = 400.285
limmat_river_high = 401.822
limmat_depth_mean = 0.7

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))
fig.suptitle("Figure 16: River and Groundwater Level Cross-Sections", fontsize=16, y=0.95)

# Plot Sihl
ru.plot_cross_section(ax1, "a) River Sihl", sihl_gw_mean, sihl_gw_high, sihl_river_mean, sihl_river_high, sihl_depth_mean)

# Plot Limmat
ru.plot_cross_section(ax2, "b) River Limmat", limmat_gw_mean, limmat_gw_high, limmat_river_mean, limmat_river_high, limmat_depth_mean)

plt.tight_layout(rect=[0, 0, 1, 0.93]) # Adjust layout to make space for suptitle
plt.show()

From the two conceptualized cross-section at the gauge location on the river Sihl we see that we are dealing with a loosing stream which is disconnected from the groundwater table. We therefore have a constant flux from the river into the aquifer. The flux is not influenced by the groundwater table in the aquifer. The flux is only influenced by the river stage and the river bed elevation.
The river Limmat is also a loosing stream at the location of the gauge. However, from the conceptualized cross-section we see that the river Limmat is very likely connected to the groundwater table. We have to account for a capillary fringe of up to 30 cm (depending on the pore size distribution of the soil matrix). The flux from the river into the aquifer is influenced by the groundwater table in the aquifer. The flux is further influenced by the river stage and the river bed elevation.

The river-aquifer interaction is limited by the hydraulic conductivity and the thickness of the clogging layer (colmation layer), a bio-geochemically highly active sediment layer which forms on river beds over time. The clogging layer is typically a few centimeters thick and has a significantly lower hydraulic conductivity than the underlying aquifer material. It therefore limits the flux from the river into the aquifer. Doppler an colleagues [\[12\]](#references) discuss the implementation of the river-aquifer interaction in their groundwater flow model of the Limmat valley aquifer. 

$$q_{riv} = \frac{K_{Schmutzschicht}}{d_{Schmutzschicht}} \cdot (H_{riv} - H_{aq}) = l_{leakage} \cdot (H_{riv} - H_{aq})$$

Where:
- $q_{riv}$ is the flux from the river into the aquifer ($m$ $s^{-1}$)
- $K_{Schmutzschicht}$ is the hydraulic conductivity of the Schmutzschicht ($m$ $s^{-1}$)
- $d_{Schmutzschicht}$ is the thickness of the clogging layer of the river bed ($m$) 
- $l_{leakage}$ is the leakage coefficient ($s^{-1}$)
- $H_{riv}$ is the river water level ($m$)
- $H_{aq}$ is the groundwater level in the aquifer ($m$)

Doppler and colleauges calibrated leakage coefficients of $1.3 \cdot 10^{-6}$ $1/s$ for the river Sihl and between $3.5 \cdot 10^{-7}$ $1/s$ and $3.5 \cdot 10^{-5}$ $1/s$ for the river Limmat.  

To simplify the first rough estimation of groundwater recharge from river infiltration, we assume disconnected streams for both rivers and an average river water level of 0.3 m to 0.5 m for the rivers Sihl and Limmat respectively, yielding the following specific fluxes from the rivers to the aquifer: 



In [ ]:
q_riv_sihl = 1.3e-6 * (sihl_river_mean - sihl_gw_mean)  # m/s
q_riv_limmat = 3.5e-6 * (limmat_river_mean - limmat_gw_mean)  # m/s

print(f"Estimated specific flux from Sihl to aquifer: {q_riv_sihl:.2e} m/s")
print(f"Estimated specific flux from Limmat to aquifer: {q_riv_limmat:.2e} m/s")

The flux from the river Limmat will be on the upper end of the range as the river is in some stretches connected to the groundwater table. The aquifer may be exfiltrating into the river in some downstream stretches far away from our focus area.     

The area of the rivers Sihl and Limmat we can estimate from the geopackage layer "Oeffentliche Oberflaechengewaesser" (see Figure 17) provided by the Department of Geoinformation of the Canton of Zurich ([https://opendata.swiss/en/dataset/offentliche-oberflachengewasser](https://opendata.swiss/en/dataset/offentliche-oberflachengewasser), accessed 2025-07-06) using QGIS. Once we have determined the model boundary and estimated the area of the river beds within the model boundary, we can estimate a recharge volume from the rivers to the aquifer. 

In [ ]:
# Download the shapes of the rivers and the locations of the gauging stations. 
gauges_file_path = download_named_file(
    name='gauges',
    data_type='gis'
)
rivers_file_path = download_named_file(
    name='rivers',
    data_type='gis'
)
plot_model_area_map(
    gw_depth_path=gw_map_path, 
    rivers_path=rivers_file_path,
    gauges_path=gauges_file_path,
    custom_title="Figure 17: Model region with surface water bodies (labeled <<Rivers>>) and groundwater gauging stations (labeled <<Gauges>>).",
)

In [ ]:
create_section_completion_marker(3)  

## Summary of Model Boundaries
We have now gathered a lot of information about the Limmat valley aquifer and its hydrological and hydrogeological properties. We can now define the flow problem we want to solve with our numerical model. We can now also define the model area and the model grid.:
- The model area is the Limmat valley aquifer, which is bounded by the river Sihl in the east and the river Limmat in the north.
- The model grid is a 2D grid with a variable thickness, based on the aquifer thickness from the GIS-browser. The grid will have a resolution of 50 m in the horizontal direction and a variable thickness in the vertical direction, based on the aquifer thickness from the GIS-browser.
- The aquifer is unconfined and the top of the aquifer is represented by the topography of the Limmat valley: No-flow boundary at the bottom of the aquifer. 
- The lateral inflow from the hills in the north and south of the Limmat valley aquifer is estimated using a water balance approach on the hill sides and distributed evenly along the inflow boundaries. It is estimated using climate data and a first approximation of 20% of the precipitation infiltrating into the aquifer: Constant or time variant pre-defined lateral inflow in the north and south. 
- The river-aquifer interaction is estimated using a 3rd type boundary condition with a head-dependent flux from the river into the aquifer. 
- The outflow boundary is a fixed-head boundary condition based on the groundwater level at piezometer 481.
- The areal recharge is estimated using the climate data from the Fluntern station and a first approximation of 10% of the precipitation infiltrating into the aquifer. At the model top we have a pre-defined constant or transient flux boundary. 

We draw the model boundary polygon manually in QGIS and export it as a geopackage layer. Given the complex geometry of aquifer thickness, this is the fastest method. We can then use this layer to define the model area in our numerical model. 

If you need instructions of how to use QGIS to draw and edit a polygon, please search for instructions to create and edit polygons in QGIS on the internet. There are many resources available.  

In [ ]:
model_boundary_path = download_named_file(
    name='model_boundary',
    data_type='gis'
)

# Get the area of the model domain
gdf_boundary = gpd.read_file(model_boundary_path)
model_area_m2 = gdf_boundary.geometry.area.sum()
model_area_km2 = model_area_m2 / 1e6  # Convert to km²
print(f"Model area: {model_area_km2:.2f} km²")  

# TODO: Add labels for the 2 rivers in the map
plot_model_area_map(
    gw_depth_path=gw_map_path, 
    rivers_path=rivers_file_path,
    gauges_path=gauges_file_path,
    model_boundary_path=model_boundary_path,
    custom_title="Figure 18: Model region with the model boundary in black, surface water bodies in dark blue (labeled <<Rivers>>), and groundwater gauging stations in red triangles (labeled <<Gauges>>).",
)

Now we cut the river shapes to the model boundary to get the area of the rivers Sihl and Limmat within the model boundary. We can then use the river area to estimate the recharge from the rivers to the aquifer.

In [ ]:
# Get the path to the river shapes and to the boundary outline
river_data_path = download_named_file(name='rivers', data_type='gis')
boundary_path = download_named_file(name='model_boundary', data_type='gis')

# Intersect the river data with the model grid and only keep the parts that are 
# inside the model boundary. 
river_gdf = gpd.read_file(river_data_path)
boundary_gdf = gpd.read_file(boundary_path)
# Clip the river data to the model boundary
river_clipped = gpd.clip(river_gdf, boundary_gdf)

# Print the column names of the clipped river data to understand its structure
# print("Clipped river data columns:")
# print(river_clipped.columns)
# Print the unique values in the 'GEWAESSERNAME' column to understand the river names
# print("\nUnique river names in the clipped data:")
# print(river_clipped['GEWAESSERNAME'].unique())

# We are interested in the river sections belonging to the rivers Sihl and Limmat. 
# We will further keep the canal in the north of the Werdinsel. This canal does 
# not have a name in the river data, but we can identify it through the geometry 
# and the unique OBJID. 
river_clipped = river_clipped[
    (river_clipped['GEWAESSERNAME'].isin(['Sihl', 'Limmat'])) |
    (river_clipped['OBJID'].isin(['32998', '34996', '37804', '95527']))
]

# Calculate the area of the clipped river sections for Sihl and Limmat
sihl_area = river_clipped[river_clipped['GEWAESSERNAME'] == 'Sihl'].geometry.area.sum()
limmat_area = river_clipped[river_clipped['GEWAESSERNAME'] == 'Limmat'].geometry.area.sum()
canal_area = river_clipped[river_clipped['OBJID'].isin(['32998', '34996', '37804', '95527'])].geometry.area.sum()

print(f"Area of Sihl within model boundary: {sihl_area:.0f} m²")
print(f"Area of Limmat within model boundary: {limmat_area:.0f} m²")
print(f"Area of canal within model boundary: {canal_area:.0f} m²")

# Plot the clipped river data to verify
fig, ax = plt.subplots(figsize=(12, 12))
river_clipped.plot(ax=ax, color='blue', linewidth=2, label='Clipped River Data')
blue_polygon = mpatches.Patch(facecolor='blue', linewidth=2, label='Clipped River Data')
boundary_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
red_line = mlines.Line2D([], [], color='red', linewidth=2, label='Model Boundary')
ax.set_title(f"Figure 19: Clipped river data within the model boundary. The areas of the river beds are: Sihl: {sihl_area:.0f} m², Limmat: {limmat_area:.0f} m², Canal: {canal_area:.0f} m²", fontsize=14)
ax.set_xlabel("X-coordinate")
ax.set_ylabel("Y-coordinate")
ax.set_aspect('equal', adjustable='box')
ax.legend(handles=[blue_polygon, red_line])
plt.show()

As the canal in the north of the Werdinsel is downstream of our area of interest, and lined with concrete, we do not consider it in our model. A first rough estimate of the leakage inflow from the rivers Sihl and Limmat to the aquifer is now:

## Groundwater pumping
The Limmat valley aquifer is heavily used for groundwater abstraction. The groundwater is used for drinking water supply but also for industrial purposes and for cooling. To estimate the total groundwater abstraction, we get the list of active and former concessions for groundwater abstractions and recharges from the GIS-browser of the Canton of Zurich. The layer is called "Wasserfassungen" and can be found in the layer list of the GIS-browser. Let's have a look at the layer in Figure 20. 

In [ ]:
# Get the path to the well locations
well_data_path = download_named_file(name='wells', data_type='gis')

# Read the geopackage into a geopandas dataframe
wells_gdf = gpd.read_file(well_data_path)

# Inspect the table
wells_gdf.head()

Let's have a look at the columns:    
- N: Northing
- E: Easting
- GWR_ID: ID of the groundwater concession (wells with the same GWR_ID are part of the same concession)
- FASSBEZ: Description of the location of the well
- FASSART: Type of the well (e.g. vertical or horizontal well)
- NUTZART: Use of the well
- BESCHREIBUNG: Description of allowed pumping amounts

GWR_ID, NUTZART, and BESCHREIBUNG are relevant for us (appart from the location information). Let's inspect the values in these columns for further analysis.

In [ ]:
# Print unique values in relevant columns
print("Unique NUTZART values:", wells_gdf['NUTZART'].unique())

NUTZART contains many cryptic abbreviations. We'll provide a mapping of these abbreviations to their full descriptions for better understanding below. The abbreviations are specific for the canton of Zurich and may be different in other regions. 

In [ ]:
# Map the unique values of NUTZART to their full descriptions
# You don't need to learn this mapping. They are just here for your reference. 
nutzart_mapping = pd.DataFrame([
    ("TW",   "Trinkwasser",                 "Drinking water"),
    ("BW",   "Brauchwasser",                "Process water / Non-potable water"),
    ("BEW",  "Bewässerung",                 "Irrigation"),
    ("GWE",  "Gewerbe",                     "Commercial use"),
    ("GWEIN","Gewerbe und Industrie",       "Commercial and industrial use"),
    ("KW",   "Kühlwasser",                  "Cooling water"),
    ("NWV",  "Natur und Wasserhaushalt / Versickerung", "Ecological / recharge use"),
    ("SA",   "Sickeranlage",                "Infiltration system"),
    ("WPG",  "Wärmepumpe Grundwasser",      "Groundwater heat pump"),
    ("WE",   "Wellness / Erholung",         "Wellness / recreational use"),
    ("WPZ",  "Wärmepumpen-Zirkulation",     "Heat pump circulation"),
], 
columns = ["Abbreviation", "German", "English"])

print(nutzart_mapping.to_string(index=False))

We are interested in larger concessions in the shallow aquifer part. We look for unique concession IDs before the "_" character in the GWR_ID column. Let's see how many unique concession IDs we have within the boundaries of our model. We'll want to restrict our search for active wells only so we remove all rows that contain the string "aufgehoben" (decommissioned) or "ungenutzt" (not used) in the column BESCHREIBUNG. 

In [ ]:
# Clip the GeoDataFrame to the model boundaries
wells_gdf = wells_gdf.clip(boundary_gdf)

# Add a concession_id (part before first "_")
wells_gdf['concession_id'] = wells_gdf['GWR_ID'].str.split('_').str[0]

# Flag decommissioned and unused (case-insensitive match)
is_decommissioned = wells_gdf['BESCHREIBUNG'].str.contains('aufgehoben', case=False, na=False)
is_unused = wells_gdf['BESCHREIBUNG'].str.contains('ungenutzt', case=False, na=False)

# Split into active / decommissioned
wells_active = wells_gdf[~is_decommissioned & ~is_unused].copy()
wells_decommissioned = wells_gdf[is_decommissioned].copy()

# Unique concession ids
all_concessions = wells_gdf['concession_id'].nunique()
active_concessions = wells_active['concession_id'].nunique()
decomm_concessions = wells_decommissioned['concession_id'].nunique()

print(f"Total concessions (within boundary): {all_concessions}")
print(f"Active concessions: {active_concessions}")
print(f"Decommissioned concessions: {decomm_concessions}")

There are 67 active concessions within the boundaries of our model. Let's see how many concessions we have in each category in column BESCHREIBUNG.  

In [ ]:
# Count the unique concession_id of active_concessions for each unique value in BESCHREIBUNG
active_concessions_count = wells_active.groupby('BESCHREIBUNG')['concession_id'].nunique()

print(f"Number of active concessions by BESCHREIBUNG: {active_concessions_count}")

Let's focus on the 10 large concessions in the model region. 

In [ ]:
id_col    = "concession_id"   # adjust if different
desc_col  = "BESCHREIBUNG"
use_col   = "NUTZART"

phrase_high      = "Grundwasserfassung mit Ertrag > 3000 l/min"
phrase_recharge  = "Grundwasseranreicherungsanlage, Rückversickerung, Sickergalerie"

# 1. Keep only wells with a non-empty NUTZART (still “active” set assumed already)
wells_valid = wells_active[
    wells_active[use_col].notna() &
    wells_active[use_col].astype(str).str.strip().ne("")
].copy()

# 2. Identify high-yield wells
mask_high = wells_valid[desc_col].str.contains(phrase_high, case=False, na=False)
high_yield_wells = wells_valid[mask_high]

# 3. Concessions having at least one high-yield well
concessions_with_high = (
    high_yield_wells[id_col]
    .dropna()
    .astype(str)
    .unique()
)

# 4. Subset ALL wells belonging to those concessions (this is the expanded set)
large_concessions = wells_valid[
    wells_valid[id_col].astype(str).isin(concessions_with_high)
].copy()

# 5. Classify well role inside those concessions
large_concessions["WELL_GROUP"] = np.select(
    [
        large_concessions[desc_col].str.contains(phrase_high, case=False, na=False),
        large_concessions[desc_col].str.contains(phrase_recharge, case=False, na=False),
    ],
    ["HighYield", "Recharge"],
    default="OtherWithinConcession"
)

# (Optional) If you want to keep only wells that are either HighYield or Recharge
# plus all other wells in those *same* concessions (already done above).
# If you instead want to drop the 'OtherWithinConcession' group, uncomment:
# large_concessions = large_concessions[large_concessions['WELL_GROUP'] != 'OtherWithinConcession'].copy()

print(f"Concessions with high-yield wells: {len(concessions_with_high)}")
print("Counts by WELL_GROUP:")
print(large_concessions["WELL_GROUP"].value_counts())


We highlight these large concessions in an interactive map. The color of the concessions indicate the NUTZART (usage type) of the well. Small active concessions are not shown in this map.  

In [ ]:
# Display large_concessions and color the dots by NUTZART. 
# The map should be interactive. Hovering over the dots should display more information about the concession.
# Add the boundary_gdf to the map and use openstreetmap as the base layer.
display_concessions_map(
    large_concessions, 
    boundary_gdf=boundary_gdf, 
    map_title="Figure 20: Wells belonging to large concessions (> 3000 l/min) in the model area colored by concession ID and marked by use type. Small active concessions are not shown in this map."
)


We see that we have intensive thermal use of groundwater in the Limmat valley aquifer (Concession use "WPG", colored in blue, orange, green and violet in Figure 20). Thermal use of groundwater is typically not a consumptive use of groundwater in the Limmat valley aquifer; it is abstracted in one well (labelled "Entnahme" in column "FASSART") and returned to the aquifer in a different well after use (labeled "Rückgabe" in column "FASSART") (you will see the FASSART if you hover your cursor over the well).  

For the purpose of this instructive model, we will not implement existing heat extraction and reinjection wells for now but concentrate on the main abstraction wells which are located in the Hardhof well field (concession b1-71, olive green in Figure 20).

Hardhof is one of 4 pillars of the drinking supply of the city of Zurich [\[13\]](#references) and contributes an average of 15 % of the annual drinking water supply of the water works (most of the citys drinking water is treated lake water). In the Hardhof well field, river bank filtrate is extracted along the river Limmat, infiltrated in 3 recharge basins and 12 infiltration wells, and finally extracted in 4 horizontal wells to be fed into the drinking water network. The Hardhof is quite complex to model and we will not be able to fully reproduce the hydrogeology there with our simplified model. The Hardhof is very interesting for us though, as they use an operational groundwater flow model that assimilated groundwater head observations to optimize the infiltration in the vertical wells in real-time. In fact, this is the site, where real-time groundwater modelling and well field control was first developed more than a decade ago. According to the last published statistical yearbook of the city of Zurich, the water works produce around 7 million cubic meters of groundwater per year [\[14\]](#references).  

Note that the concession is for 104'000 l/min (GIS-Browser). Only about 13% of the concessioned pumping is actually extracted. The concession only gives the upper limit of what is actually extracted from groundwater. 

> **Suggestion:**. 
> If you are located in Zurich, we highly encourage you to visit the Hardhof well field in one of their free and public guided tours.  

Let's also have a look at the medium sized concessions so we don't miss important fluxes. We'll neglect the small sized concessions as their pumping lies below 300 l/min which is well within our uncertainty range.

In [ ]:
id_col    = "concession_id"   # adjust if different
desc_col  = "BESCHREIBUNG"
use_col   = "NUTZART"

phrase_medium      = "Grundwasserfassung mit Ertrag 300 - 3000 l/min"
phrase_recharge  = "Grundwasseranreicherungsanlage, Rückversickerung, Sickergalerie"

# 1. Keep only wells with a non-empty NUTZART (still “active” set assumed already)
wells_valid = wells_active[
    wells_active[use_col].notna() &
    wells_active[use_col].astype(str).str.strip().ne("")
].copy()

# 2. Identify medium-yield wells
mask_medium = wells_valid[desc_col].str.contains(phrase_medium, case=False, na=False)
medium_yield_wells = wells_valid[mask_medium]

# 3. Concessions having at least one medium-yield well
concessions_with_medium = (
    medium_yield_wells[id_col]
    .dropna()
    .astype(str)
    .unique()
)

# 4. Subset ALL wells belonging to those concessions (this is the expanded set)
medium_concessions = wells_valid[
    wells_valid[id_col].astype(str).isin(concessions_with_medium)
].copy()

# 5. Classify well role inside those concessions
medium_concessions["WELL_GROUP"] = np.select(
    [
        medium_concessions[desc_col].str.contains(phrase_medium, case=False, na=False),
        medium_concessions[desc_col].str.contains(phrase_recharge, case=False, na=False),
    ],
    ["MediumYield", "Recharge"],
    default="OtherWithinConcession"
)

# (Optional) If you want to keep only wells that are either HighYield or Recharge
# plus all other wells in those *same* concessions (already done above).
# If you instead want to drop the 'OtherWithinConcession' group, uncomment:
# medium_concessions = medium_concessions[medium_concessions['WELL_GROUP'] != 'OtherWithinConcession'].copy()

print(f"Concessions with medium-yield wells: {len(concessions_with_medium)}")
print("Counts by WELL_GROUP:")
print(medium_concessions["WELL_GROUP"].value_counts())

# Count the number of concessions for TW and BW
print("Number of concessions for TW and BW:")
print(medium_concessions[medium_concessions[use_col].isin(["TW", "BW", "TW, BW"])][id_col].nunique())

display_concessions_map(
    medium_concessions, 
    boundary_gdf=boundary_gdf, 
    map_title="Figure 21: Wells belonging to medium concessions (300 - 3000 l/min) in the model area colored by concession ID and marked by use type. Small and large active concessions are not shown in this map."
)


Most of the medium sized concessions again concern heat extraction for district heating systems which consist of extraction & reinjection of groundwater, so no significant consumptive use is expected. There are 5 concessions for drinking water and industrial water use (TW and BW). Let's have a close look at them.  

In [ ]:
# List the active wells in medium concessions which are used for drinking water and industrial water use
active_wells_medium_concessions = medium_concessions[medium_concessions[use_col].isin(["TW", "BW", "TW, BW"])]
print(active_wells_medium_concessions[['GWR_ID', 'FASSBEZ', 'FASSART', 'NUTZART']])

By clicking on the well in the GIS-Browser, we see slightly more information than in the geodata layer we have available here. The Lochergut concession (b010063) is for 520 l/min, and the Schlachthof concession is for 370 l/min. The concessioned pumping rate for concession number b010113 is 2000 l/min. The 2 drinking water wells in the north of the Limmat (concession numbers n010039_01 and n010085_01) allow pumping of 1200 l/min and 3000 l/min, respectively.   

As the two drinking water wells in the north of the river will most likely pump river water infiltrate and are far downstream of our focus area, we'll not implement them for now. As we saw with the Hardhof, only a fraction of the concessioned amount is typically extracted, let's assume 40% for now. Then, the medium sized concessions in the south of the Limmat would amount to about 600'000 m3/year (less than 10 % of the Hardhof abstractions).   

In [ ]:
outflow_pumping = 7600000  # m3/year

# Since we don't know the exact groundwater abstractions, we'll also have to 
# estimate the uncertainty of our estimate. 
# We'll assume a conservative uncertainty of 20%.

## Summary of Perceptual Model

Let's update the perceptual model with the information we have gathered so far (Figure 19).

In [ ]:
# Display figure with updated fluxes

In [ ]:
# Calculate the missing fluxes for the water balance
net_recharge = 0.2 * model_area_km2 * 1000000  # m³/year, assuming 200 mm/year net recharge
river_sihl_inflow = sihl_area * q_riv_sihl * 365 * 24 * 3600  # m³/year
river_limmat_inflow = limmat_area * q_riv_limmat * 365 * 24 * 3600  # m³/year
gw_outflow_west = (outflow_lower_bound + outflow_upper_bound) / 2 * 365  # m³/year, average of lower and upper bound

# Create a summary table with the most relevant in- and outflows of the model domain
inflow_summary = {
    "Component": [
        "Net areal recharge to the aquifer (R)",
        "Lateral inflow from the south",
        "Lateral inflow from the north",
        "River Sihl (net inflow)",
        "River Limmat (net inflow)",
    ], 
    "Value (10^6 m³/year)": [
        round(net_recharge / 1e6, 1),
        round(inflow_south / 1e6, 1),
        round(inflow_north / 1e6, 1),
        round(river_sihl_inflow / 1e6, 1),
        round(river_limmat_inflow / 1e6, 1)
    ]
}
outflow_summary = {
    "Component": [
        "Groundwater outflow in the west",
        "Groundwater pumping"
    ], 
    "Value (10^6 m³/year)": [
        round(gw_outflow_west / 1e6, 1),
        round(outflow_pumping / 1e6, 1)
    ]
}

inflow_summary_df = pd.DataFrame(inflow_summary)
outflow_summary_df = pd.DataFrame(outflow_summary)

print("Inflows:")
print(inflow_summary_df)
print("")
print("Outflows:")
print(outflow_summary_df)
print("")
print(f"Sum of inflows: {inflow_summary_df['Value (10^6 m³/year)'].sum():.1f} 10^6 m³/year")
print(f"Sum of outflows: {outflow_summary_df['Value (10^6 m³/year)'].sum():.1f} 10^6 m³/year")
print(f"Difference (inflows - outflows): {(inflow_summary_df['Value (10^6 m³/year)'].sum() - outflow_summary_df['Value (10^6 m³/year)'].sum()):.1f} 10^6 m³/year")

We currently have much higher inflows than outflows but that's fine for this first estimate of the fluxes. We did this exercise to get a rough idea of the groundwater budget in the Limmat valley aquifer. We already know that the river-aquifer interaction is complicated and that we cannot adequately estimate this flux with the data we have currently at hand.

One term is still missing in the water balance equation: the change in storage. We need to account for the groundwater that is being stored or released in the aquifer over time. This term is crucial for understanding the overall dynamics of the groundwater system and will be included in the final model.

In [ ]:
create_section_completion_marker(4)  

## Monitoring the Limmat Valley Aquifer
To get a first idea about the groundwater levels in the Limmat valley aquifer, we will have a look at the groundwater map.

Several authorities do groundwater monitoring in the Limmat valley aquifer. We will start with the federal office for the environment (FOEN) which maintains a network of groundwater observation wells. You find an overview over the available groundwater monitoring sites at [https://map.geo.admin.ch/](https://map.geo.admin.ch/) in layer *Groundwater level/spring discharge* [\[13\]](#references). One monitoring well maintained by FOEN is located in the Limmat valley aquifer but far downstream of our area of interest in the city center. Further, this well is a drinking water production well and can therefore not be used as an outflow boundary.  

Few groundwater observation wells are operated by the cantonal office of the environment (AWEL) [\[14\]](#references) (Figure 19).

In [ ]:
du.display_image(
    image_filename='GWmonitoring_locations_AWEL.png', 
    image_folder='1_perceptual_model',
    caption='Figure 19: Monitoring wells in the Limmat valley aquifer. Yearbook sheets for each site can be accessed through the popup window from each site. Source: https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html, accessed 2025-05-01.'
)

> **🤔 Think about it**  
> What are the major hydrological processes in the Limmat valley aquifer?  
> Where would you set the boundaries of the Limmat valley aquifer?

Now let's see if we can detect a storage change in the aquifer over time. We do this by comparing the groundwater levels in the observation wells over time. We can use the groundwater level data from the cantonal monitoring wells to do this.

We have downloaded the groundwater level data from the cantonal monitoring wells and can now analyze it to identify any trends or changes in storage over time. If you are interested in 


In [ ]:
# Download the data if it doesn't exist
gw_ts_path = download_named_file(
    name='groundwater_timeseries',
    data_type='time_series'
)

# Import the data
gw_ts_raw = pd.read_csv(gw_ts_path)
# Only keep relevant columns
gw_ts = gw_ts_raw[['date', 'value', 'well_id']]
# Make sure date is in a recognized date format
gw_ts['date'] = pd.to_datetime(gw_ts['date'], errors='coerce')
# Drop duplicates
gw_ts = gw_ts.drop_duplicates()
# Ensure sorted by time
gw_ts = gw_ts[['date', 'well_id', 'value']].sort_values('date')
# The coordinates of IDs B5-3 and B53 mark the same location (albeit in 
# different coordinate systems). We'll use the newer name B5-3 for both.
gw_ts.loc[gw_ts['well_id'] == 'B53', 'well_id'] = 'B5-3'


In [ ]:
# Plot values against date, grouped by well_id
plt.figure(figsize=(12, 6))
for well_id, group in gw_ts.groupby('well_id'):
    plt.plot(group['date'], group['value'], label=well_id)
plt.xlabel('Date')
plt.ylabel('Groundwater Level (m.a.s.l.)')
plt.title('Groundwater Levels Over Time by Well')
plt.legend()
plt.grid()
plt.show()

Let's zoom in to the standardized time series to see if the system is in steady state. 

In [ ]:
# Normalize the values by well_id
gw_ts_std = gw_ts.copy()
gw_ts_std['value'] = gw_ts_std.groupby('well_id')['value'].transform(lambda x: (x - x.mean()) / x.std())

# Monthly mean per well (month start periods). Produces NaN for months with no data.
monthly = (
    gw_ts_std.set_index('date')
      .groupby('well_id')['value']
      .resample('MS')   # use 'M' for month-end instead
      .mean()
      .reset_index()
      .rename(columns={'value': 'value_monthly'})
)

# Annual mean per well
annual = (
    gw_ts_std.set_index('date')
      .groupby('well_id')['value']
      .resample('AS')   # use 'A' for year-end instead
      .mean()
      .reset_index()
      .rename(columns={'value': 'value_annual'})
)

# Create the figure
fig = px.line(
    annual,
    x='date',
    y='value_annual',
    color='well_id',
    render_mode='webgl',           # GPU-accelerated for many points
)

# Add range slider + quick selectors for past periods
fig.update_xaxes(
    rangeslider=dict(visible=True),
    rangeselector=dict(
        buttons=list([
            dict(count=5,  label='5Y',  step='year',  stepmode='backward'),
            dict(count=10,  label='10Y',  step='year', stepmode='backward'),
            dict(count=20,  label='20Y',  step='year',  stepmode='backward'),
            dict(step='all', label='All')
        ])
    )
)

fig.update_layout(
    title='Figure X: St. Groundwater Levels Over Time by Well',
    xaxis_title='Date',
    yaxis_title='Standardized Groundwater Level (m.a.s.l.)',
    hovermode='x unified',
    height=520,
    legend_title_text='Well',
    margin=dict(l=40, r=10, t=50, b=40)
)

fig.show()

Looking at the past 20 years of average groundwater table data, no significant trends are visible. We can assume that the groundwater levels have remained relatively stable over this period and that the system is in steady state.

$$ \Delta S = 0 $$

In [ ]:
create_section_completion_marker(5)  

## References
[\[1\]](#the-limmat-valley-aquifer) Hug J., and Beilick, A. (1934): Die Grundwasserverhältnisse des Kantons Zürich. In: Beiträge zur Geologie der Schweiz - Geotechnische Serie - Hydrologie. German. Available online here: https://scnat.ch/de/uuid/i/0bd7aa3b-0bd7-5d54-9e2f-597a42dada50-Die_Grundwasserverh%C3%A4ltnisse_des_Kantons_Z%C3%BCrich (accessed 2025-05-01)     
[\[2\]](#the-limmat-valley-aquifer) Doppler, T., Hendricks Franssen, H.-J., Kaiser H.-P., Kuhlman U., Stauffer, F. (2007): Field evidence of a dynamic leakage coefficient for modelling river–aquifer interactions. Journal of Hydrology, Volume 347, Issues 1–2, DOI: https://doi.org/10.1016/j.jhydrol.2007.09.017.    
[\[3\]](#the-limmat-valley-aquifer) GIS-browser of the canton of Zurich: https://www.gis.zh.ch/ (accessed 2025-05-01)    
[\[4\]](#top-topography) Federal Office of Topography swisstopo, DHM25 height model, published 2024-01-08, [https://www.swisstopo.admin.ch/en/height-model-dhm25](https://www.swisstopo.admin.ch/en/height-model-dhm25) (accessed 2025-05-01)    
[\[5\]](#lateral-model-boundaries) Freeze, R. A., and Cherry, J. A. (1979): Groundwater. Prentice-Hall, Englewood Cliffs, New Jersey, USA. ISBN: 978-0133653122. Open-book available here: [https://gw-project.org/books/groundwater/](https://gw-project.org/books/groundwater/)   
[\[6\]](#climate) MeteoSwiss: https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year (accessed 2025-05-01)  
[\[7\]](#climate) Opendata.swiss: Klimanormwerte, publisher: Federal Office of Meterology and Climatology. [https://opendata.swiss/en/dataset/klimanormwerte](https://opendata.swiss/en/dataset/klimanormwerte) (accessed 2025-05-01)  
[\[8\]](#climate) Qui, Guo Yu, Yan, Chunhua, Liu, Yuanbo (2023): Urban evapotranspiration and its effects on water budget and energy balance: Review and perspectives. Earth-Science Reviews, Volume 246. DOI: [https://doi.org/10.1016/j.earscirev.2023.104577](https://doi.org/10.1016/j.earscirev.2023.104577)  
[\[9\]](#lateral-inflow-from-hills) Lerner, David N. (2002): Identifying and quantifying urban recharge: a review. Hydrogeology Journal, Volume 10, Issue 1, pp 143–152. DOI: [https://doi.org/10.1007/s10040-001-0177-7](https://doi.org/10.1007/s10040-001-0177-1)    
[\[10\]](#river-discharge) Locations of hydrological gauging stations maintained by the Federal Office for the Environment (FOEN): https://map.geo.admin.ch (accessed 2025-05-01)  
[\[11\]](#river-discharge) Locations of hydrological gauging stations maintained by the cantonal office for waste, water, energy, and air (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/abfluss-wasserstand.html (accessed 2025-05-01)   
[\[12\]](#river-aquifer-interaction) Doppler, T., Hendricks Franssen, H.-J., Kaiser H.-P., Kuhlman U., Stauffer, F. (2007): Field evidence of a dynamic leakage coefficient for modelling river–aquifer interactions. Journal of Hydrology, Volume 347, Issues 1–2, DOI: https://doi.org/10.1016/j.jhydrol.2007.09.017.   
[\[13\]](#groundwater-pumping) City of Zurich, Drinking Water Supply: [https://www.stadt-zuerich.ch/de/umwelt-und-energie/wasser.html](https://www.stadt-zuerich.ch/de/umwelt-und-energie/wasser.html) (accessed 2025-08-21)    
[\[14\]](#groundwater-pumping) Statistisches Jahrbuch der Stadt Zürich 2017 (Statistical yearbook of the city of Zurich 2017), Kapitel Wasser und Energie (Chapter water and energy), page 8: Wasserabgabe nach Wasserherkunft; Grundwasser (water deliveries by origin; groundwater).  [https://www.stadt-zuerich.ch/de/politik-und-verwaltung/statistik-und-daten/publikationen-und-dienstleistungen/publikationen/jahrbuch.html](https://www.stadt-zuerich.ch/de/politik-und-verwaltung/statistik-und-daten/publikationen-und-dienstleistungen/publikationen/jahrbuch.html) (accessed 2025-08-21)      
[\[13\]](#monitoring-the-limmat-valley-aquifer) Locations of groundwater monitoring wells maintained by Federal Office for the Environment (FOEN): https://www.zh.ch/de/umwelt-und-natur/wasser/grundwasser/monitoring.html (accessed 2025-05-01)  
[\[14\]](#monitoring-the-limmat-valley-aquifer) Cantonal office of the environment (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html (accessed 2025-05-01)
